In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import shapefile as shp
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import sys
sys.path.append('../scripts')
from preprocess_script import count_outliers

In [ ]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Dataset Joining")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "9g") 
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("OFF")

# Dataset downloaded from https://data.gov.au/dataset/ds-dga-2c79581f-600e-4560-80a8-98adb1922dfc/details?q=correspondence%20asgs, data dictionary also in the same.

Data to be put into tables/correspondence/

In [ ]:
# Dataset downloaded from https://data.gov.au/dataset/ds-dga-2c79581f-600e-4560-80a8-98adb1922dfc/details?q=correspondence%20asgs, data dictionary also in the same.

correspondence_df = spark.read.csv('../data/tables/correspondence/CG_POSTCODE_2021_SA2_2021.csv', header=True)
correspondence_df.show()


preferred_SA2 = correspondence_df.groupBy('POSTCODE').agg(max('RATIO_FROM_TO'))
preferred_SA2.show()


preferred_SA2 = preferred_SA2.withColumnRenamed("max(RATIO_FROM_TO)", "RATIO_FROM_TO")


preferred_SA2.show()
preferred_SA2.count()
preferred_SA2.dropna()
preferred_SA2.count()


preferred_SA2 = preferred_SA2.join(correspondence_df, on=["POSTCODE", "RATIO_FROM_TO"], how = "left")


preferred_SA2.show()


preferred_SA2.select('POSTCODE', 'SA2_CODE_2021')

In [ ]:
# Saving the correspondence
preferred_SA2.write.mode('overwrite').parquet('../data/curated/correspondence/')